In [1]:
import os
import shutil
import random

# Carpetas de origen
IMAGES_DIR = "../data/images"  # Imágenes originales limpias
MASKS_DIR = "../data/masks"  # Máscaras limpias

# Carpetas de destino
BASE_OUTPUT_DIR = "../data/"
TRAIN_DIR = os.path.join(BASE_OUTPUT_DIR, "train")
VAL_DIR = os.path.join(BASE_OUTPUT_DIR, "val")
TEST_DIR = os.path.join(BASE_OUTPUT_DIR, "test")

# Crear subcarpetas si no existen
for split_dir in [TRAIN_DIR, VAL_DIR, TEST_DIR]:
    os.makedirs(os.path.join(split_dir, "images"), exist_ok=True)
    os.makedirs(os.path.join(split_dir, "masks"), exist_ok=True)

# Parámetros del split
train_ratio = 0.70
val_ratio = 0.15
test_ratio = 0.15

# Asegurarnos de que sumen 1 (opcional)
assert abs(train_ratio + val_ratio + test_ratio - 1.0) < 1e-6, "Ratios must sum to 1."

# Listar nombres de archivos de imágenes
# Asumiendo que las imágenes siguen un patrón: 0000_image.jpg / 0000_image.png
all_images = [f for f in os.listdir(IMAGES_DIR) if "_image" in f]
# Ordenar para consistencia (opcional)
all_images.sort()

print("Total imágenes encontradas:", len(all_images))

# Emparejar con sus máscaras, asumiendo que comparten el índice
# Ej: 0000_image.jpg -> 0000_mask.png
def get_mask_name(img_name):
    # Reemplazamos "_image" por "_mask" y ajustamos la extensión
    # Si tus máscaras están en png siempre, forzamos la extensión ".png"
    return img_name.replace("_image", "_mask").rsplit(".", 1)[0] + ".png"

paired_files = []
for img_name in all_images:
    mask_name = get_mask_name(img_name)
    img_path = os.path.join(IMAGES_DIR, img_name)
    msk_path = os.path.join(MASKS_DIR, mask_name)
    
    # Verificar que la máscara exista
    if os.path.exists(msk_path):
        paired_files.append((img_name, mask_name))
    else:
        print(f"Mask not found for {img_name}, skipping.")

print(f"Total pares válidos imagen-máscara: {len(paired_files)}")

# Mezclar aleatoriamente
random.shuffle(paired_files)

# Calcular tamaños de cada split
total_files = len(paired_files)
train_count = int(total_files * train_ratio)
val_count = int(total_files * val_ratio)
test_count = total_files - train_count - val_count  # para asegurar que sume correctamente

# Dividir
train_pairs = paired_files[:train_count]
val_pairs = paired_files[train_count:train_count+val_count]
test_pairs = paired_files[train_count+val_count:]

print(f"Train: {len(train_pairs)}")
print(f"Val:   {len(val_pairs)}")
print(f"Test:  {len(test_pairs)}")

# Función para copiar o mover archivos
def copy_files(pairs, split_dir):
    for img_name, msk_name in pairs:
        src_img = os.path.join(IMAGES_DIR, img_name)
        src_msk = os.path.join(MASKS_DIR, msk_name)
        
        dst_img = os.path.join(split_dir, "images", img_name)
        dst_msk = os.path.join(split_dir, "masks", msk_name)
        
        # Copiamos (shutil.copy2 conserva metadatos; usa move si prefieres)
        shutil.copy2(src_img, dst_img)
        shutil.copy2(src_msk, dst_msk)

# Copiar cada split
copy_files(train_pairs, TRAIN_DIR)
copy_files(val_pairs, VAL_DIR)
copy_files(test_pairs, TEST_DIR)

print("Split completado con éxito.")

Total imágenes encontradas: 149
Total pares válidos imagen-máscara: 149
Train: 104
Val:   22
Test:  23
Split completado con éxito.
